Semantic chunking involves taking the embeddings of every sentence in the document, comparing the similarity of all sentences with each other, and then grouping sentences with the most similar embeddings together.

In [12]:
!pip install -qU langchain_experimental langchain_openai langchain_community langchain ragas chromadb langchain-groq fastembed pypdf openai

In [13]:
!wget "https://arxiv.org/pdf/1810.04805.pdf"

--2024-09-05 13:08:37--  https://arxiv.org/pdf/1810.04805.pdf
Resolving arxiv.org (arxiv.org)... 151.101.195.42, 151.101.131.42, 151.101.3.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.195.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/1810.04805 [following]
--2024-09-05 13:08:37--  http://arxiv.org/pdf/1810.04805
Connecting to arxiv.org (arxiv.org)|151.101.195.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 775166 (757K) [application/pdf]
Saving to: ‘1810.04805.pdf.1’

1810.04805.pdf.1    100%[===================>] 757.00K  --.-KB/s    in 0.07s   

2024-09-05 13:08:37 (10.0 MB/s) - ‘1810.04805.pdf.1’ saved [775166/775166]



In [14]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
#
loader = PyPDFLoader("1810.04805.pdf")
documents = loader.load()
#
print(len(documents))

16


## Perform Native Chunking(RecursiveCharacterTextSplitting)

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
    length_function=len,
    is_separator_regex=False)\

naive_chunks = text_splitter.split_documents(documents)
for chunk in naive_chunks[10:15]:
  print(chunk.page_content+ "\n")

BERT BERT 
E[CLS] E1 E[SEP] ... ENE1’... EM’
C
T1
T[SEP] ...
 TN
T1’...
 TM’
[CLS] Tok 1 [SEP] ... Tok NTok 1 ... TokM 
Question Paragraph Start/End Span 
BERT 
E[CLS] E1 E[SEP] ... ENE1’... EM’
C
T1
T[SEP] ...
 TN
T1’...
 TM’
[CLS] Tok 1 [SEP] ... Tok NTok 1 ... TokM 
Masked Sentence A Masked Sentence B 
Pre-training Fine-Tuning NSP Mask LM Mask LM 
Unlabeled Sentence A and B Pair SQuAD 
Question Answer Pair NER MNLI Figure 1: Overall pre-training and ﬁne-tuning procedures for BERT. Apart from output layers, the same architec-
tures are used in both pre-training and ﬁne-tuning. The same pre-trained model parameters are used to initialize
models for different down-stream tasks. During ﬁne-tuning, all parameters are ﬁne-tuned. [CLS] is a special
symbol added in front of every input example, and [SEP] is a special separator token (e.g. separating ques-
tions/answers).
ing and auto-encoder objectives have been used
for pre-training such models (Howard and Ruder,

2018; Radford et al., 201

In [16]:
from langchain.embeddings.fastembed import FastEmbedEmbeddings
embed_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [17]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(embed_model, breakpoint_threshold_type="percentile")
semantic_chunks = semantic_chunker.create_documents([d.page_content for d in documents])

for semantic_chunk in semantic_chunks:
  if "Effect of Pre-training Tasks" in semantic_chunk.page_content:
    print(semantic_chunk.page_content)
    print(len(semantic_chunk.page_content))


Dev Set
Tasks MNLI-m QNLI MRPC SST-2 SQuAD
(Acc) (Acc) (Acc) (Acc) (F1)
BERT BASE 84.4 88.4 86.7 92.7 88.5
No NSP 83.9 84.9 86.5 92.6 87.9
LTR & No NSP 82.1 84.3 77.5 92.1 77.8
+ BiLSTM 82.1 84.1 75.7 91.6 84.9
Table 5: Ablation over the pre-training tasks using the
BERT BASE architecture. “No NSP” is trained without
the next sentence prediction task. “LTR & No NSP” is
trained as a left-to-right LM without the next sentence
prediction, like OpenAI GPT. “+ BiLSTM” adds a ran-
domly initialized BiLSTM on top of the “LTR + No
NSP” model during ﬁne-tuning. ablation studies can be found in Appendix C. 5.1 Effect of Pre-training Tasks
We demonstrate the importance of the deep bidi-
rectionality of BERT by evaluating two pre-
training objectives using exactly the same pre-
training data, ﬁne-tuning scheme, and hyperpa-
rameters as BERT BASE :
No NSP : A bidirectional model which is trained
using the “masked LM” (MLM) but without the
“next sentence prediction” (NSP) task. LTR & No NSP : A left

In [18]:
from langchain_community.vectorstores import Chroma
semantic_chunk_vectorstore = Chroma.from_documents(semantic_chunks, embedding=embed_model)

We will “limit” our semantic retriever to k = 1 to demonstrate the power of the semantic chunking strategy while maintaining similar token counts between the semantic and naive retrieved context.

In [19]:
semantic_chunk_retriever = semantic_chunk_vectorstore.as_retriever(search_kwargs={"k" : 1})

In [20]:
semantic_chunk_retriever.invoke("Describe the Feature-based Approach with BERT?")

[Document(page_content='The right part of the paper represents the\nDev set results. For the feature-based approach,\nwe concatenate the last 4 layers of BERT as the\nfeatures, which was shown to be the best approach\nin Section 5.3. From the table it can be seen that ﬁne-tuning is\nsurprisingly robust to different masking strategies. However, as expected, using only the M ASK strat-\negy was problematic when applying the feature-\nbased approach to NER. Interestingly, using only\nthe R NDstrategy performs much worse than our\nstrategy as well.')]

In [21]:
from langchain_core.prompts import ChatPromptTemplate

rag_template = """\
Use the following context to answer the user's query. If you cannot answer, please respond with 'I don't know'.

User's Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)

In [22]:
from groq import Groq
from langchain_groq import ChatGroq

chat_model = ChatGroq(temperature=0,
                      model_name="mixtral-8x7b-32768",
                      api_key="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",)


In [23]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

semantic_rag_chain = (
    {"context":semantic_chunk_retriever, "question":RunnablePassthrough()}
    | rag_prompt
    | chat_model
    | StrOutputParser()
)

In [24]:
semantic_rag_chain.invoke("Describe the Feature-based Approach with BERT?")

'The Feature-based Approach with BERT involves concatenating the last 4 layers of BERT as features, which was found to be the most effective method in Section 5.3 of the paper. This approach is then compared in the table on the right side of the paper, which presents the Dev set results. Fine-tuning is shown to be robust to different masking strategies, but using only the MASK strategy is problematic when applying the feature-based approach to Named Entity Recognition (NER). Using only the RND strategy performs significantly worse than the proposed strategy.'

In [25]:
semantic_rag_chain.invoke("What is SQuADv2.0?")

'SQuAD v2.0, or Squad 2.0, is a version of the Stanford Question Answering Dataset (SQuAD) that extends the problem definition of SQuAD 1.1 by allowing for the possibility that no short answer exists in the provided paragraph. This makes the problem more realistic. To extend the SQuAD v1.1 BERT model for this task, questions that do not have an answer are treated as having an answer span with start and end at the [CLS] token. The probability space for the start and end answer span positions is extended to include the position of the [CLS] token. For prediction, the score of the no-answer span is compared to the score of the best non-null span. The TriviaQA data used for this task consists of paragraphs from TriviaQA-Wiki formed of the first 400 tokens in documents, that contain at least one of the provided possible answers.'

## Implement a RAG pipeline using Naive Chunking Strategy

In [26]:
naive_chunk_vectorstore = Chroma.from_documents(naive_chunks, embedding=embed_model)
naive_chunk_retriever = naive_chunk_vectorstore.as_retriever(search_kwargs={"k" : 3})


In [27]:
naive_rag_chain = (
    {"context" : naive_chunk_retriever, "question" : RunnablePassthrough()}
    | rag_prompt
    | chat_model
    | StrOutputParser()
)

In [28]:
naive_rag_chain.invoke("Describe the Feature-based Approach with BERT?")

'The feature-based approach with BERT involves extracting the activations from one or more layers of the pre-trained BERT model without fine-tuning any of its parameters. These contextual embeddings are then used as input to a separately initialized two-layer BiLSTM before the classification layer. The feature-based approach has been shown to perform competitively with state-of-the-art methods in Named Entity Recognition (NER) tasks. Specifically, the best performing feature-based method concatenates the token representations from the top four hidden layers of the pre-trained Transformer. This method is only 0.3 F1 behind fine-tuning the entire model, demonstrating the effectiveness of BERT for both fine-tuning and feature-based approaches. In the context of NER tasks, the feature-based approach concatenates the last 4 layers of BERT as the features, which was shown to be the best approach. Fine-tuning has been found to be surprisingly robust to different masking strategies, but using 

In [29]:
naive_rag_chain.invoke("What is SQuADv2.0?")

'SQuADv2.0, or Squad 2.0, is a version of the Stanford Question Answering Dataset (SQuAD) that extends the problem definition of SQuAD 1.1 by allowing for the possibility that no short answer exists in the provided paragraph. This makes the problem more realistic. To handle such questions, a simple approach is used to extend the SQuAD v1.1 BERT model for this task. Questions that do not have an answer are treated as having an answer span with start and end at the [CLS] token, and the probability space for the start and end answer span positions is extended to include the position of the [CLS] token. Prediction is then made by comparing the score of the no-answer span to the score of the best non-null span. The document also mentions that systems are not allowed to use any public data when training, and modest data augmentation is used by ﬁne-tuning on TriviaQA before ﬁne-tuning on SQuAD.'

## Ragas Assessment Comparison for Semantic Chunker

In [30]:
synthetic_data_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
    length_function=len,
    is_separator_regex=False
)
#
synthetic_data_chunks = synthetic_data_splitter.create_documents([d.page_content for d in documents])
print(len(synthetic_data_chunks))

72


In [31]:
questions = []
ground_truths_semantic = []
contexts = []
answers = []

question_prompt = """\
You are a teacher preparing a test. Please create a question that can be answered by referencing the following context.

Context:
{context}
"""
question_prompt = ChatPromptTemplate.from_template(question_prompt)

ground_truth_prompt = """\
Use the following context and question to answer this question using *only* the provided context.

Question:
{question}

Context:
{context}
"""
ground_truth_prompt = ChatPromptTemplate.from_template(ground_truth_prompt)
question_chain = question_prompt | chat_model | StrOutputParser()
ground_truth_chain = ground_truth_prompt | chat_model | StrOutputParser()
for chunk in synthetic_data_chunks[10:20]:
  questions.append(question_chain.invoke({"context" : chunk.page_content}))
  contexts.append([chunk.page_content])
  ground_truths_semantic.append(ground_truth_chain.invoke({"question" : questions[-1], "context" : contexts[-1]}))
  answers.append(semantic_rag_chain.invoke(questions[-1]))

In [32]:
from datasets import load_dataset, Dataset

qagc_list = []

for question, answer, context, ground_truth in zip(questions, answers, contexts, ground_truths_semantic):
  qagc_list.append({
      "question" : question,
      "answer" : answer,
      "contexts" : context,
      "ground_truth" : ground_truth
  })

eval_dataset = Dataset.from_list(qagc_list)
eval_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 10
})

In [33]:
eval_dataset[0]

{'question': 'Question: Based on the given context, what are the special symbols added in front of every input example and separating questions/answers in BERT?\n\nAnswer: The special symbols added in front of every input example in BERT is [CLS], and [SEP] is the special separator token used to separate questions/answers.',
 'answer': 'The special symbols added in front of every input example in BERT is [CLS], and [SEP] is the special separator token used to separate questions/answers.',
 'contexts': ['BERT BERT \nE[CLS] E1 E[SEP] ... ENE1’... EM’\nC\nT1\nT[SEP] ...\n TN\nT1’...\n TM’\n[CLS] Tok 1 [SEP] ... Tok NTok 1 ... TokM \nQuestion Paragraph Start/End Span \nBERT \nE[CLS] E1 E[SEP] ... ENE1’... EM’\nC\nT1\nT[SEP] ...\n TN\nT1’...\n TM’\n[CLS] Tok 1 [SEP] ... Tok NTok 1 ... TokM \nMasked Sentence A Masked Sentence B \nPre-training Fine-Tuning NSP Mask LM Mask LM \nUnlabeled Sentence A and B Pair SQuAD \nQuestion Answer Pair NER MNLI Figure 1: Overall pre-training and ﬁne-tuning p

In [34]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

In [1]:
from ragas import evaluate

result = evaluate(
    eval_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
     llm=chat_model,
    embeddings=embed_model,
    raise_exceptions=False
)

NameError: name 'eval_dataset' is not defined